In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exl-health-ml-hackathon/sample_submission.csv
/kaggle/input/exl-health-ml-hackathon/train.csv
/kaggle/input/exl-health-ml-hackathon/test.csv


In [96]:
train = pd.read_csv('/kaggle/input/exl-health-ml-hackathon/train.csv')
test = pd.read_csv('/kaggle/input/exl-health-ml-hackathon/test.csv')

In [97]:
x = train.isnull().sum().reset_index(name = 'count')
cols_remove = list(x[x['count'] >= 4000]['index'])
cols_remove

['feature_15', 'feature_29', 'feature_34', 'feature_35']

In [98]:
train = train.drop(columns = cols_remove)
test = test.drop(columns = cols_remove)

In [99]:
category_columns = list(train.select_dtypes(['object']).columns)
numeric_columns = list(train.select_dtypes(['float64']).columns)
print("Numeric Columns : ", numeric_columns)
print("Categorical Columns: ", category_columns)

Numeric Columns :  ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_32', 'feature_33']
Categorical Columns:  ['feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_30', 'feature_31']


In [100]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.impute import KNNImputer 
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier 
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
import category_encoders as ce

In [101]:
x = train.drop(columns = ['level','id'])
y = train[['level']]

In [102]:
for i in x.columns:
    x[i] = x[i].astype(str)

In [103]:
# class_distribution = {0: 11818, 1: 6878, 2: 868, 3: 4174}

# # Function to create an XGBClassifier with the appropriate scale_pos_weight for each class
# def create_weighted_xgb(class_index, class_distribution):
#     pos_count = class_distribution[class_index]
#     neg_count = sum(class_distribution.values()) - pos_count
#     scale_pos_weight = neg_count / pos_count
    
#     # Return an XGBClassifier with the computed scale_pos_weight
#     return XGBClassifier(
#         objective='multi:softprob',
#         scale_pos_weight=scale_pos_weight,
#         use_label_encoder=False,
#         eval_metric="mlogloss"
#     )

# # Preprocessor setup with KNNImputer and TargetEncoder
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', Pipeline([
#             ('encoder', ce.TargetEncoder()), 
#             ('imputer', KNNImputer(n_neighbors=3))
#         ]), x.columns)
#     ]
# )

# # Create an OvR classifier with dynamically calculated scale_pos_weight
# classifiers = [
#     create_weighted_xgb(class_index, class_distribution)
#     for class_index in class_distribution
# ]

# # Define the pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', OneVsRestClassifier(estimator=XGBClassifier(
#         use_label_encoder=False, eval_metric="mlogloss", 
#     )))
# ])

In [110]:
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import KNNImputer
# from sklearn.multiclass import OneVsRestClassifier
# from catboost import CatBoostClassifier
# import category_encoders as ce
# import numpy as np

# # Class distribution
# class_distribution = {0: 11818, 1: 6878, 2: 868, 3: 4174}

# # Function to calculate scale_pos_weight for each binary classifier in the One-vs-Rest setup
# def calculate_scale_pos_weight(class_index, class_distribution):
#     pos_count = class_distribution[class_index]
#     neg_count = sum(class_distribution.values()) - pos_count
#     return neg_count / pos_count if pos_count > 0 else 1

# # Function to create a CatBoostClassifier with scale_pos_weight
# def create_weighted_catboost(class_index, class_distribution):
#     scale_pos_weight = calculate_scale_pos_weight(class_index, class_distribution)
#     return CatBoostClassifier(
#         loss_function='Logloss',  # Binary classification for each OvR classifier
#         learning_rate=0.01,
#         iterations=2000,
#         od_type="Iter",
#         early_stopping_rounds=500,
#         random_seed=24,
#         silent=True,
#         task_type="CPU",
#         scale_pos_weight=scale_pos_weight
#     )

# # Preprocessor setup with TargetEncoder and KNNImputer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', Pipeline([
#             ('encoder', ce.TargetEncoder()), 
#             ('imputer', KNNImputer(n_neighbors=3))
#         ]), x.columns)
#     ]
# )

# # Define OneVsRestClassifier using CatBoostClassifier for each class with its respective scale_pos_weight
# one_vs_rest_classifier = OneVsRestClassifier(
#     estimator=CatBoostClassifier(
#         loss_function='Logloss',  # Use binary classification for OvR
#         learning_rate=0.01,
#         iterations=2000,
#         od_type="Iter",
#         early_stopping_rounds=500,
#         random_seed=24,
#         silent=True,
#         task_type="CPU"
#     )
# )

# # Define the pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', one_vs_rest_classifier)
# ])

In [119]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                        test_size=0.2,
                                                        random_state=43,
                                                        stratify=y)

In [122]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
import category_encoders as ce

# Class distribution (replace with your actual data)
class_distribution = {0: 11818, 1: 6878, 2: 868, 3: 4174}

# Function to calculate scale_pos_weight for each class
def calculate_scale_pos_weight(class_index, class_distribution):
    pos_count = class_distribution[class_index]
    neg_count = sum(class_distribution.values()) - pos_count
    return neg_count / pos_count if pos_count > 0 else 1

# Preprocessor setup with TargetEncoder and KNNImputer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('encoder', ce.TargetEncoder()), 
            ('imputer', KNNImputer(n_neighbors=3))
        ]), x.columns)
    ]
)

# Set up CatBoostClassifier with appropriate settings
catboost_estimator = CatBoostClassifier(
    loss_function='Logloss',  
    task_type="CPU",
    learning_rate=0.01,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=24,
    silent=True
)

# Define Stratified KFold cross-validation
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Arrays to store out-of-fold predictions and test predictions
oof_preds_1 = np.zeros((x_train.shape[0], len(np.unique(y_train))))  # Model 1 OOF
oof_preds_2 = np.zeros((x_train.shape[0], len(np.unique(y_train))))  # Model 2 OOF
test_preds_1 = np.zeros((x_test.shape[0], len(np.unique(y_train)), n_splits))  # Model 1 Test predictions
test_preds_2 = np.zeros((x_test.shape[0], len(np.unique(y_train)), n_splits))  # Model 2 Test predictions

# Stratified KFold Loop for model training and OOF predictions
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    X_tr, X_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    preprocessor.fit(X_tr, y_tr)
    # Fit first model (Model 1)
    model_1 = CatBoostClassifier(loss_function='Logloss', learning_rate=0.01, iterations=2000, early_stopping_rounds=500, task_type="CPU")
    model_1.fit(X_tr, y_tr, eval_set=[preprocessor.transform(X_val, y_val)], verbose=False)
    
    # Store OOF predictions for Model 1
    oof_preds_1[val_idx] = model_1.predict_proba(X_val)
    
    # Predict on test set for Model 1
    test_preds_1[:, :, fold_idx] = model_1.predict_proba(x_test)

    # Fit second model (Model 2) with different settings or model (if applicable)
    model_2 = CatBoostClassifier(loss_function='Logloss', learning_rate=0.05, iterations=2000, early_stopping_rounds=500, task_type="CPU")
    model_2.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    # Store OOF predictions for Model 2
    oof_preds_2[val_idx] = model_2.predict_proba(X_val)
    
    # Predict on test set for Model 2
    test_preds_2[:, :, fold_idx] = model_2.predict_proba(x_test)

# Generate the final test predictions by averaging the test predictions across all folds
final_test_preds_1 = np.mean(test_preds_1, axis=2)
final_test_preds_2 = np.mean(test_preds_2, axis=2)


TypeError: ColumnTransformer.transform() takes 2 positional arguments but 3 were given

In [111]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                        test_size=0.2,
                                                        random_state=43,
                                                        stratify=y)

In [115]:
pipeline.fit(x, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder()),
                                                                  ('imputer',
                                                                   KNNImputer(n_neighbors=3))]),
                                                  Index(['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_30', 'feature_31', 'feature_32',
       'feature_33'],
      dtype='object'))])),
                ('classifier',
                 OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f7eafd15990>))])

In [116]:
final_pred = pipeline.predict(x_test)

In [117]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, final_pred))


              precision    recall  f1-score   support

           0       0.79      0.89      0.84      2364
           1       0.74      0.68      0.71      1376
           2       0.98      0.26      0.41       173
           3       0.71      0.65      0.68       835

    accuracy                           0.77      4748
   macro avg       0.80      0.62      0.66      4748
weighted avg       0.77      0.77      0.76      4748



In [118]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Arrays to store out-of-fold predictions and test predictions
oof_preds = np.zeros((x_train.shape[0], len(np.unique(y_train))))
test_preds = np.zeros((x_test.shape[0], len(np.unique(y_train)), n_splits))

# Loop over each fold
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    # Split data into training and validation sets
    x_tr, x_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Fit the pipeline on the training set of the current fold
    pipeline.fit(x_tr, y_tr)
    
    # Predict on the validation set and store in OOF array
    oof_preds[val_idx] = pipeline.predict_proba(x_val)
    
    # Predict on the test set and store predictions for averaging later
    test_preds[:, :, fold_idx] = pipeline.predict_proba(x_test)

# Calculate the OOF CV score
oof_score = f1_score(y_train, np.argmax(oof_preds, axis=1), average="weighted")
print("Out-of-Fold CV F1 Score:", oof_score)

# Average the test predictions across all folds
final_test_preds = np.mean(test_preds, axis=2)

# Final predictions on the test set
final_predictions = np.argmax(final_test_preds, axis=1)
print("Final Test Predictions:", final_predictions)

Out-of-Fold CV F1 Score: 0.6923895701399382
Final Test Predictions: [0 0 1 ... 1 0 1]


In [79]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
import category_encoders as ce
import numpy as np
from scipy.stats import uniform, randint

# Define class distribution
class_distribution = {0: 11818, 1: 6878, 2: 868, 3: 4174}

# Function to create an XGBClassifier with the appropriate scale_pos_weight for each class
def create_weighted_xgb(class_index, class_distribution):
    pos_count = class_distribution[class_index]
    neg_count = sum(class_distribution.values()) - pos_count
    scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1
    return XGBClassifier(
        objective='binary:logistic',
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        eval_metric="mlogloss"
    )

# Preprocessor setup with TargetEncoder, KNNImputer, and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('encoder', ce.TargetEncoder()), 
            ('imputer', KNNImputer(n_neighbors=3)),
            ('scaler', StandardScaler())
        ]), x.columns)
    ]
)

# Base model for OneVsRestClassifier
xgb_estimator = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Hyperparameter space for tuning
param_dist = {
    "classifier__estimator__learning_rate": uniform(0.01, 0.3),
    "classifier__estimator__n_estimators": randint(50, 200),
    "classifier__estimator__max_depth": randint(3, 10),
    "classifier__estimator__subsample": uniform(0.6, 0.4),
    "classifier__estimator__colsample_bytree": uniform(0.6, 0.4)
}

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', OneVsRestClassifier(estimator=xgb_estimator))
])

# StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV to optimize hyperparameters
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=50,
    scoring='f1_weighted',
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV on the training data
random_search.fit(x_train, y_train)

# Display best parameters and best weighted F1 score
print("Best Parameters:", random_search.best_params_)
print("Best Weighted F1 Score:", random_search.best_score_)

# Final predictions on the test set
y_pred = random_search.predict(x_test)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'classifier__estimator__colsample_bytree': 0.942529716751237, 'classifier__estimator__learning_rate': 0.13135243813665703, 'classifier__estimator__max_depth': 3, 'classifier__estimator__n_estimators': 165, 'classifier__estimator__subsample': 0.6336559859980195}
Best Weighted F1 Score: 0.6947832324519552


In [80]:
 params = {'colsample_bytree': 0.942529716751237, 'learning_rate': 0.13135243813665703, 'max_depth': 3, 'n_estimators': 165, 'subsample': 0.6336559859980195}

In [85]:
class_distribution = {0: 11818, 1: 6878, 2: 868, 3: 4174}

# Function to create an XGBClassifier with the appropriate scale_pos_weight for each class
def create_weighted_xgb(class_index, class_distribution):
    pos_count = class_distribution[class_index]
    neg_count = sum(class_distribution.values()) - pos_count
    scale_pos_weight = neg_count / pos_count
    
    # Return an XGBClassifier with the computed scale_pos_weight
    return XGBClassifier(
        objective='multi:softprob',
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        eval_metric="mlogloss"
    )

# Preprocessor setup with KNNImputer and TargetEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('encoder', ce.TargetEncoder()), 
            ('imputer', KNNImputer(n_neighbors=3))
        ]), x.columns)
    ]
)

# Create an OvR classifier with dynamically calculated scale_pos_weight
classifiers = [
    create_weighted_xgb(class_index, class_distribution)
    for class_index in class_distribution
]

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', OneVsRestClassifier(estimator=XGBClassifier(**params
        ,use_label_encoder=False, eval_metric="mlogloss" 
    )))
])

In [88]:
pipeline.fit(x,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder()),
                                                                  ('imputer',
                                                                   KNNImputer(n_neighbors=3))]),
                                                  Index(['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_16',
       'f...
                                                             grow_policy=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.13135243813665703,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=3,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=165,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=None, ...)))])

In [90]:
from sklearn.metrics import accuracy_score, classification_report
final_pred = pipeline.predict(x_train)
print(classification_report(y_train, final_pred))

              precision    recall  f1-score   support

           0       0.76      0.87      0.81      9454
           1       0.69      0.62      0.65      5502
           2       0.73      0.13      0.22       695
           3       0.66      0.60      0.63      3339

    accuracy                           0.72     18990
   macro avg       0.71      0.56      0.58     18990
weighted avg       0.72      0.72      0.71     18990



In [91]:
pred = pipeline.predict(test)